In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
#calculating CPM

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)
df.drop(columns=['total_revenue'], inplace=True)
df = df[df['CPM'] >= 0]
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
train = df[df['date'] < '2019-06-22']
test = df[df['date'] >= '2019-06-22']
test_up = test['CPM'].quantile(0.95)
test = test[test['CPM'] < test_up ] # remove outliers
train = train[train['CPM'] < test_up] # remove outliers (from train too)
train.shape, test.shape

In [ ]:
df['date'].describe()

## Lets look on data

In [ ]:
train.info()

In [ ]:
pd.concat([train,test]).groupby('date').agg({'CPM':'mean'}).plot()

In [ ]:
num_cols = ['total_impressions', 'viewable_impressions', 'measurable_impressions', 'CPM']

In [ ]:
train[num_cols].corr().style.background_gradient(cmap='inferno').set_precision(2)

In [ ]:
train[num_cols].describe()

In [ ]:
train.columns

In [ ]:
cat_cols = ['site_id', 'ad_type_id', 'geo_id', 'device_category_id',
       'advertiser_id', 'order_id', 'line_item_type_id', 'os_id',
       'integration_type_id', 'monetization_channel_id', 'ad_unit_id']

## Build model

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
bad_cols = ['date', 'CPM']

X = train.drop(columns=bad_cols)
y = train['CPM']

model = CatBoostRegressor()
model.fit(X, y, 
          cat_features=cat_cols, 
#           silent=True
         )

In [ ]:
y_train_pred = model.predict(X)
train_score = mean_squared_error(y, y_train_pred)
print(f"TRAIN MSE : {train_score:.3f}")

In [ ]:
plt.bar(x=model.feature_names_, height=model.feature_importances_)
model.feature_names_

In [ ]:
X_test = test.drop(columns=bad_cols)
y_test = test['CPM']

y_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"TEST MSE: {test_mse}")

> ***HACHU ZACHET***